# 데이터의 차원 
- 차원 축소의 목표
    - 훈련 속도, 데이터 시각화에 유용
- 원리
    - 거의 대부분 무관(일정)한 데이터 => 제거 대상
    - 일부는 특성간 강한 연관성 => 표현 대상
- 배경
    - 고차원일수록 인접 두 샘플의 거리가 멀어진다.
    - 따라서 데이터가 고차원일수록 많은 외삽이 일어나 과대적합됨
- 개선 방법
    - 훈련 샘플의 밀도가 충분히 높아질 때까지 데이터 추가(차원 대비 기하급수적으로 필요)
    - 데이터 차원 축소

# Principal Components Analysis
- 공부 필요..
https://github.com/rickiepark/handson-ml2/blob/master/08_dimensionality_reduction.ipynb

- 개념
    - 데이터에 가장 가까운 초평면을 찾아 투영하면 데이터의 차원을 줄일 수 있다.
- 가정
    - 데이터의 평균이 0이다.(pca 하기전 데이터를 원점에 맞춰야함)
- 초평면 찾기
    - 분산이 가장 큰 축(원본과 투영본 사이의 평균 제곱거리가 최소화되는 축)을 선택한다.
    - 그 축에 직교하고 다음으로 분산이 큰 축을 선택한다.
    - 데이터 셋 차원 수 n만큼 반복한다.
    - d차원 초평면(주성분 n개 중 d개=공분산 행렬 첫 d열)에 투영하여 데이터 차원 축소가능
- 초평면 빠르게 찾기
    - 특잇값 분해 
    - 공분산 행렬의 고유벡터=분산이 어느 방향으로 가장 큰 가=주성분
   

In [1]:
#3D 데이터셋을 만듭니다
import numpy as np
np.random.seed(4)
m = 60
w1, w2 = 0.1, 0.3
noise = 0.1

angles = np.random.rand(m) * 3 * np.pi / 2 - 0.5
X = np.empty((m, 3))
X[:, 0] = np.cos(angles) + np.sin(angles)/2 + noise * np.random.randn(m) / 2
X[:, 1] = np.sin(angles) * 0.7 + noise * np.random.randn(m) / 2
X[:, 2] = X[:, 0] * w1 + X[:, 1] * w2 + noise * np.random.randn(m)

In [2]:
from sklearn.decomposition import PCA

#데이터셋에서 평균을 빼는 작업도 대신 처리해 줍니다
# PCA변환기의 components_ 속성에 W^d 전치(계산된 주성분)가 담겨있습니다
pca= PCA(n_components=2)
X2D= pca.fit_transform(X)

In [4]:
# 각 주선분의 축을 따라 있는 데이터셋의 분산 비율
pca.explained_variance_ratio_

array([0.84248607, 0.14631839])

# 축소할 차원 수 고르기
- 충분한 분산(ex 95%)가 될 때까지 차원수 늘리기
- 설명된 분산을 차원수에 대한 함수로 그려 파악하기

In [5]:
# 보존하려는 분산의 비율 지정 가능
pca = PCA(n_components=0.95)
X_reduced= pca.fit_transform(X)

# 재구성
- 재구성 오차: 원본과 재구성 데이터 사이의 평균 제곱 거리

# 속도/메모리 개선
- 랜덤 PCA
    - 확률적 알고리즘을 사용해 처음 d개 주성분에 대한 근삿값을 빠르게 찾는다.
- 점진적 PCA
    - SVD알고리즘은 전체 훈련세트를 필요로함 IPCA는 추가되는 데이터에 대해 특정순간의 배열 일부만 사용해 학습(partial_fit)

In [2]:
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
import numpy as np

mnist = fetch_openml('mnist_784', version=1, as_frame=False)
mnist.target = mnist.target.astype(np.uint8)

X = mnist["data"]
y = mnist["target"]

X_train, X_test, y_train, y_test = train_test_split(X, y)

#pca = PCA(n_components=154)
#rnd_pca= PCA(n_components=154, svd_solver='randomized')
#X_reduced= pca.fit_transform(X_train)
#X_recovered= pca.inverse_transform(X_reduced)

In [9]:
from sklearn.decomposition import IncrementalPCA

n_batches= 100
inc_pca= IncrementalPCA(n_components=154)
for X_batch in np.array_split(X_train, n_batches):
    inc_pca.partial_fit(X_batch)
    
X_reduced= inc_pca.transform(X_train)

#  Projection - 커널 PCA
- 5장 커널트릭
    - 매우 높은 고차원공간(특성 공간)으로 암묵적으로 매핑하여 SVM의 비선형 분류/회귀 가능
    - 고차원 특성공간에서의 선형 결정 경계 = 원본공간에서는 복잡한 비선형 결정경계
- kPCA
    - 동일한 성질을 이용한 비지도 학습
    - 그리드 탐색으로 가장 성능이 좋은 커널과 하이퍼 파라미터 gamma를 찾는다.(best_params_)
        - 재구성 원상의 오차를 최소화하는 커널과 하이퍼파라미터 선택
- fit_inverse_transform 
    - 투영된 샘플=훈련세트, 원본 샘플=타깃세트로 하여 훈련

In [4]:
from sklearn.decomposition import KernelPCA
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline

clf= Pipeline([
    ("kpca", KernelPCA(n_components=2)),
    ("log_reg", LogisticRegression())
])

param_grid=[{
    "kpca__gamma": np.linspace(0.03, 0.05, 10),
    "kpca__kernel": ["rbf", "sigmoid"]
}]

grid_search = GridSearchCV(clf, param_grid, cv=3)
grid_search.fit(X,y)

print(grid_search.best_params_)

KeyboardInterrupt: 

In [ ]:
from sklearn.metrics import mean_squared_error

rbf_pca=KernelPCA(n_components=2, kernel="rbf",gamma=0.0433, fit_inverse_transform=True)
X_reduced=rbf_pca.fit_transform(X)
X_preimage= rbf_pca.inverse_transform(X_reduced)

mean_squared_error(X_X_preimage)

# Manifold - LLE
- 지역 선형 임베딩(Locally linear embedding)
    - 투영에 의존하지안흔ㄴ 매니폴드 학습법
- 과정
    - 각 훈련샘플에 가장 가까운 이웃들과의 선형관계(가중치) 측정
    - 국부관계가 가장 잘 보존되는 훈련세트의 저차원 표현을 찾는다.
        - d차원 공간에서 x_i의 상 p_i라면
        - 선형관계를 적용하여 예측값과 오차가 최소화될 것이라는 가정
- 원본 특성 차원수^2에 복잡도 비례하여 대용량 데이터에 적용하기 어려움

In [ ]:
from sklearn.manifold import LocallyLinearEmbedding

lle= LocallyLinearEmbedding(n_components=2, n_neighbors=10)
X_reduced=lle.fit_transform(X)